In [25]:
import os
from data.load_data import data_load
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import matplotlib.dates as mdates
import pingouin as pg
from torch.utils.data import Dataset, ConcatDataset, DataLoader
import scipy

In [40]:
df_survey = data_load(data_keys={"surveys"}, wave=7)['surveys']
# df_birth = data_load(data_keys={"birth"}, wave=5)['birth']
TARGET_SYMPTOM = 'fatigue'
identifier = 'record_id'
dataset_dir_name = 'dataset_LT'

/storage/home/ruizhu/Rui-s_BUMP_code_dump/src/s3_utils.py:68: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, compression=compression)


In [41]:
question_148_level = {"Yes": 0, "No": 0}
question_149_to_157_level = {"Never": 1, 'Sometimes': 2, 'Regularly': 3, 'Often': 4, 'Always': 5}
question_158_level = {"Never": 5, 'Sometimes': 4, 'Regularly': 3, 'Often': 2, 'Always': 1}
disabled_level = {"Never": 0, 'Sometimes': 0, 'Regularly': 0, 'Often': 0, 'Always': 0}


In [42]:
# for FAS
question_id_lst = list(range(148, 159))
num_questions = len(list(range(148, 159)))
question_levels_lst = [question_149_to_157_level] * num_questions
question_levels_lst[0] = question_148_level
question_levels_lst[4] = question_158_level
question_levels_lst[-1] = question_158_level

# # disable mental questions
# for q in range(6, 11):
#     question_levels_lst[q] = disabled_level
# # disable physical questions
# for q in range(1, 6):
#     question_levels_lst[q] = disabled_level

question_level_mapping_dict = dict(zip(question_id_lst, question_levels_lst))

In [43]:
def map_answer_to_int(row):
    curr_level_mapping = question_level_mapping_dict[row['question_id']]
    return curr_level_mapping[row['answer_text']]


In [44]:
if TARGET_SYMPTOM == "fatigue":
    weekly_feature_name = "fas_score"
    df_weekly_survey = df_survey.loc[df_survey['title'] =='Fatigue survey']
    df_weekly_survey[weekly_feature_name] = df_weekly_survey.apply(map_answer_to_int, axis=1)
    grouped_df_weekly_survey = df_weekly_survey.groupby([identifier, 'date'])[weekly_feature_name].sum().reset_index()
    
elif TARGET_SYMPTOM == "edema":
    weekly_feature_name = 'pitting_edema'
    grouped_df_weekly_survey = data_load(data_keys={'pitting_edema'})['pitting_edema'][[identifier, 'date', 
                                                                                       'pitting_edema']]
    grouped_df_weekly_survey = grouped_df_weekly_survey.dropna(subset=['pitting_edema'])
    
grouped_df_weekly_survey['date'] = pd.to_datetime(grouped_df_weekly_survey['date'])
grouped_df_weekly_survey['week_year'] = grouped_df_weekly_survey['date'].dt.strftime('%Y-%U')
print(grouped_df_weekly_survey)

      record_id       date  fas_score week_year
0     direct_10 2022-05-21         28   2022-20
1     direct_10 2022-06-03         24   2022-22
2     direct_10 2022-06-10         29   2022-23
3     direct_10 2022-06-17         27   2022-24
4     direct_10 2022-07-01         27   2022-26
...         ...        ...        ...       ...
9255  sema4_992 2023-03-15         33   2023-11
9256  sema4_992 2023-03-22         32   2023-12
9257  sema4_993 2022-08-04         24   2022-31
9258  sema4_993 2022-08-10         21   2022-32
9259  sema4_993 2022-08-31          0   2022-35

[9260 rows x 4 columns]


/tmp/ipykernel_15252/1197895825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weekly_survey[weekly_feature_name] = df_weekly_survey.apply(map_answer_to_int, axis=1)


In [45]:
# grouped_df_weekly_survey
for i in range(1):
    print(i)


0


In [46]:
# curr_user_fas[curr_user_fas['date'] == '2021-09-21']

In [47]:

def get_previous_dates(date, day_range=7):
    week_dates = [(pd.to_datetime(date) - timedelta(days=i)) for i in range(day_range)]
    return week_dates

class SameDayMappingDataset(Dataset):
    def __init__(self, dataset_x, dataset_y, user_id, feature_name_lst_x, feature_name_lst_y, train_time_steps, target_time_steps, one_week_back=True, normalize=True, 
                return_user_id=False, dict_format=False):
        self.user_id = user_id
        self.feature_name_lst_x = feature_name_lst_x
        self.feature_name_lst_y = feature_name_lst_y
        self.one_week_back = one_week_back
        self.train_time_steps = train_time_steps
        self.target_time_steps = target_time_steps
        self.return_user_id = return_user_id
        self.dict_format = dict_format
        if normalize:
            self.dataset_x = self._normalize(dataset_x, feature_name_lst_x, x_true=True)
            self.dataset_y = self._normalize(dataset_y, feature_name_lst_y, x_true=False)
        else:
            self.dataset_x = dataset_x
            self.dataset_y = dataset_y
        # 0 impute

        mean_value = 0
        self.dataset_x = self.dataset_x.fillna(value=mean_value)

    def _normalize(self, df, feature_names, x_true):
        

        if x_true:
            dates_with_one_week = []
            for date in self.train_time_steps:
                if self.one_week_back:
                    week_dates = get_previous_dates(date)
                # print(week_dates)
                else:
                    week_dates = get_previous_dates(date, 1)
                dates_with_one_week.extend(week_dates)
                
            train_dates = [date.strftime('%Y-%m-%d') for date in dates_with_one_week]
        else:
            train_dates = self.train_time_steps
        # print(len(train_dates))
        train_indices = df.index[df['date'].isin(train_dates)]
        # print('len(df)', len(df))
        assert len(train_indices) != 0
        # print('train_indices', train_indices)
        def min_max_normalize_subset(feature):
            # print('len(feature)', len(feature))
            min_val = feature.iloc[train_indices].min()
            max_val = feature.iloc[train_indices].max()
            if min_val == max_val:
                max_val += 1
            return (feature - min_val) / (max_val - min_val)
        # print(feature_names)
        df[feature_names] = df[feature_names].apply(min_max_normalize_subset)
        return df
    
    def __len__(self):
        return len(self.target_time_steps)
    def set_dict_format(self, boolean):
        self.dict_format = boolean
    def set_return_user_id(self, boolean):
        self.return_user_id = boolean
    
    
    def __getitem__(self, idx):
        time_step = self.target_time_steps[idx]
        # print(time_step)
        # print(self.dataset_y)
        y = self.dataset_y[self.dataset_y['date'] == time_step][self.feature_name_lst_y].values

        # print(y.shape)
        if self.one_week_back:
            prev_week_dates = get_previous_dates(time_step)
            x = self.dataset_x[self.dataset_x['date'].isin(prev_week_dates)][self.feature_name_lst_x].values
            desired_size = [7, 1]
        else:
            prev_week_dates = get_previous_dates(time_step, 1)
            x = self.dataset_x[self.dataset_x['date'].isin(prev_week_dates)][self.feature_name_lst_x].values
            desired_size = [1, 1]
        # print('x.shape', x.shape)
        
        padding = []
        for i in range(len(desired_size)):
            pad_length = desired_size[i] - x.shape[i]
            padding.append((0, pad_length))
        x = np.pad(x, padding, mode='constant')
        
        if self.dict_format:
            if self.return_user_id:
                return {"X": x, "Y": y, "user_id": self.user_id}
            else:
                return {"X": x, "Y": y}
        else:
            if self.return_user_id:
                return x, y, self.user_id
            else:
                return x, y

In [48]:
# train_set = SameDayMappingDataset(dataset_y = curr_user_fas, dataset_x = curr_user_daily_df, 
#     user_id=curr_user, feature_name_lst_y=['fas_score'], feature_name_lst_x=['fatigue'], train_time_steps=train_times, target_time_steps=test_times)

In [49]:
os.path.exists('../dataset')


True

In [50]:

train_set_lst = []
test_set_lst = []
for curr_user in grouped_df_weekly_survey[identifier].unique():
    # curr_user = 28
    print("curr_user", curr_user)
    curr_user_fas = grouped_df_weekly_survey[grouped_df_weekly_survey[identifier] == curr_user].reset_index()
    fas_time = list(curr_user_fas['date'].unique())
    # TODO: need to generate using clean_survey_data.ipynb
    daily_csv_path = f'../{dataset_dir_name}/{TARGET_SYMPTOM}/user_{curr_user}_{TARGET_SYMPTOM}_label.csv'
    if not os.path.exists(daily_csv_path):
        print(f"path dne: {daily_csv_path}")
        continue
    curr_user_daily_df = pd.read_csv(daily_csv_path).groupby("date", as_index = False).first()[['date', TARGET_SYMPTOM]]
    curr_user_daily_df['date'] = pd.to_datetime(curr_user_daily_df['date'])
    
    daily_time = list(curr_user_daily_df['date'].unique())
    available_time = list(set(daily_time).intersection(fas_time))
    available_time = pd.to_datetime(available_time)
    available_time = [date.strftime('%Y-%m-%d') for date in available_time]
    
    # train_times = available_time[:int(len(available_time) * 0.8)]
    train_times = available_time
    
    test_times = available_time[int(len(available_time) * 0.8):]
    
    if TARGET_SYMPTOM == 'fatigue':
        threshold = 3
    elif TARGET_SYMPTOM == 'edema':
        threshold = 0
    if len(train_times) <= threshold:
        print("Too few training times")
        continue
    # if X need to be the full week before Y, i.e., Y is a survey asking about the past week
    if TARGET_SYMPTOM == 'fatigue':
        one_week_back = True
    elif TARGET_SYMPTOM == 'edema':
        one_week_back = False
    train_set = SameDayMappingDataset(dataset_y = curr_user_fas, dataset_x = curr_user_daily_df, 
    user_id=curr_user, feature_name_lst_y=[weekly_feature_name], feature_name_lst_x=[TARGET_SYMPTOM], 
                                      train_time_steps=train_times, target_time_steps=train_times, one_week_back=one_week_back)
    test_set = SameDayMappingDataset(dataset_y = curr_user_fas, dataset_x = curr_user_daily_df, 
    user_id=curr_user, feature_name_lst_y=[weekly_feature_name], feature_name_lst_x=[TARGET_SYMPTOM], 
                                     train_time_steps=train_times, target_time_steps=test_times, one_week_back=one_week_back)
    print("training_set len", len(train_set))

    train_set_lst.append(train_set)
    test_set_lst.append(test_set)
    # print("\n")
# train_set = ConcatDataset(train_set_lst)
# test_set = ConcatDataset(test_set_lst)
# train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)

curr_user direct_10
training_set len 24
curr_user direct_109
training_set len 13
curr_user direct_113
training_set len 33
curr_user direct_114
training_set len 15
curr_user direct_116
training_set len 27
curr_user direct_118
training_set len 31
curr_user direct_122
training_set len 20
curr_user direct_127
training_set len 13
curr_user direct_13
training_set len 8
curr_user direct_130
training_set len 5
curr_user direct_133
training_set len 7
curr_user direct_134
Too few training times
curr_user direct_135
training_set len 17
curr_user direct_141
training_set len 6
curr_user direct_143
training_set len 29
curr_user direct_144
training_set len 17
curr_user direct_145
training_set len 19
curr_user direct_148
training_set len 4
curr_user direct_15
training_set len 32
curr_user direct_152
training_set len 29
curr_user direct_156
Too few training times
curr_user direct_159
training_set len 18
curr_user direct_162
training_set len 15
curr_user direct_166
Too few training times
curr_user direc

In [52]:
corr_lst = []
p_value_lst = []
reliable_train_ds_lst = []
reliable_test_ds_lst = []
icc_lst = []
user_lst = []
for i, ds in enumerate(train_set_lst):
    ds.set_return_user_id(True)
    curr_loader = DataLoader(ds, batch_size=128, shuffle=True)
    for batch_features, batch_labels, user_id in curr_loader:
        daily_survey = batch_features.numpy().squeeze()
        weekly_survey = batch_labels.numpy().squeeze()
        selected_daily_survey = daily_survey#[:, -3:]
        print('selected_daily_survey.shape', selected_daily_survey.shape)
        print('weekly_survey.shape', weekly_survey.shape)
        
        # take average if we have more than one day in X
        if len(selected_daily_survey.shape) > 1:
            selected_daily_survey = np.average(selected_daily_survey, axis=1)
#         print(avg_daily_survey.shape)
#         print(weekly_survey.shape)
        my_idx = list(range(0, weekly_survey.shape[0]))
        curr_df = pd.DataFrame({'daily': selected_daily_survey, 'weekly': weekly_survey, 'target': my_idx})
        melted_df = pd.melt(curr_df, id_vars='target', var_name='y_label', value_name='y_value')
        icc = pg.intraclass_corr(data=melted_df, targets='target', raters='y_label', ratings='y_value')
        icc.set_index('Type')
        two_way_mixed_icc = icc.loc[2, 'ICC']
        icc_lst.append(two_way_mixed_icc)
        
        corr, p_value = scipy.stats.pearsonr(selected_daily_survey, weekly_survey)
        # some participants answer all 0 in their daily survey, resulting in nan value

        if two_way_mixed_icc > 0.5:
#         if True:
            reliable_train_ds_lst.append(train_set_lst[i])
            reliable_test_ds_lst.append(test_set_lst[i])
            user_lst.append(user_id[0])
            
        corr_lst.append(corr)
        p_value_lst.append(p_value)
# print(np.sum(np.array(corr_lst) > 0))
# print(np.sum(np.array(icc_lst) > 0))


selected_daily_survey.shape (24, 7)
weekly_survey.shape (24,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (33, 7)
weekly_survey.shape (33,)
selected_daily_survey.shape (15, 7)
weekly_survey.shape (15,)
selected_daily_survey.shape (27, 7)
weekly_survey.shape (27,)
selected_daily_survey.shape (31, 7)
weekly_survey.shape (31,)
selected_daily_survey.shape (20, 7)
weekly_survey.shape (20,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (8, 7)
weekly_survey.shape (8,)
selected_daily_survey.shape (5, 7)
weekly_survey.shape (5,)
selected_daily_survey.shape (7, 7)
weekly_survey.shape (7,)
selected_daily_survey.shape (17, 7)
weekly_survey.shape (17,)
selected_daily_survey.shape (6, 7)
weekly_survey.shape (6,)
selected_daily_survey.shape (29, 7)
weekly_survey.shape (29,)
selected_daily_survey.shape (17, 7)
weekly_survey.shape (17,)
selected_daily_survey.shape (19, 7)
weekly_survey.shape (19,)
selected_daily_s

/h/ruizhu/.conda/envs/pt-2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/h/ruizhu/.conda/envs/pt-2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


selected_daily_survey.shape (6, 7)
weekly_survey.shape (6,)
selected_daily_survey.shape (24, 7)
weekly_survey.shape (24,)
selected_daily_survey.shape (11, 7)
weekly_survey.shape (11,)
selected_daily_survey.shape (68, 7)
weekly_survey.shape (68,)
selected_daily_survey.shape (9, 7)
weekly_survey.shape (9,)
selected_daily_survey.shape (6, 7)
weekly_survey.shape (6,)
selected_daily_survey.shape (45, 7)
weekly_survey.shape (45,)
selected_daily_survey.shape (37, 7)
weekly_survey.shape (37,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (66, 7)
weekly_survey.shape (66,)
selected_daily_survey.shape (27, 7)
weekly_survey.shape (27,)
selected_daily_survey.shape (35, 7)
weekly_survey.shape (35,)
selected_daily_survey.shape (41, 7)
weekly_survey.shape (41,)
selected_daily_survey.shape (19, 7)
weekly_survey.shape (19,)
selected_daily_survey.shape (11, 7)
weekly_survey.shape (11,)
selected_daily_survey.shape (11, 7)
weekly_survey.shape (11,)
selected_daily

/h/ruizhu/.conda/envs/pt-2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


selected_daily_survey.shape (24, 7)
weekly_survey.shape (24,)
selected_daily_survey.shape (19, 7)
weekly_survey.shape (19,)
selected_daily_survey.shape (11, 7)
weekly_survey.shape (11,)
selected_daily_survey.shape (8, 7)
weekly_survey.shape (8,)
selected_daily_survey.shape (9, 7)
weekly_survey.shape (9,)


...

selected_daily_survey.shape (19, 7)
weekly_survey.shape (19,)
selected_daily_survey.shape (9, 7)
weekly_survey.shape (9,)
selected_daily_survey.shape (23, 7)
weekly_survey.shape (23,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (18, 7)
weekly_survey.shape (18,)
selected_daily_survey.shape (16, 7)
weekly_survey.shape (16,)
selected_daily_survey.shape (35, 7)
weekly_survey.shape (35,)
selected_daily_survey.shape (23, 7)
weekly_survey.shape (23,)
selected_daily_survey.shape (14, 7)
weekly_survey.shape (14,)
selected_daily_survey.shape (4, 7)
weekly_survey.shape (4,)
selected_daily_survey.shape (9, 7)
weekly_survey.shape (9,)
selected_daily_survey.shape (12, 7)
weekly_survey.shape (12,)
selected_daily_survey.shape (24, 7)
weekly_survey.shape (24,)
selected_daily_survey.shape (28, 7)
weekly_survey.shape (28,)
selected_daily_survey.shape (18, 7)
weekly_survey.shape (18,)
selected_daily

/h/ruizhu/.conda/envs/pt-2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


selected_daily_survey.shape (6, 7)
weekly_survey.shape (6,)
selected_daily_survey.shape (11, 7)
weekly_survey.shape (11,)
selected_daily_survey.shape (36, 7)
weekly_survey.shape (36,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)


/h/ruizhu/.conda/envs/pt-2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (32, 7)
weekly_survey.shape (32,)
selected_daily_survey.shape (16, 7)
weekly_survey.shape (16,)
selected_daily_survey.shape (6, 7)
weekly_survey.shape (6,)
selected_daily_survey.shape (14, 7)
weekly_survey.shape (14,)
selected_daily_survey.shape (19, 7)
weekly_survey.shape (19,)
selected_daily_survey.shape (27, 7)
weekly_survey.shape (27,)
selected_daily_survey.shape (29, 7)
weekly_survey.shape (29,)
selected_daily_survey.shape (44, 7)
weekly_survey.shape (44,)
selected_daily_survey.shape (24, 7)
weekly_survey.shape (24,)
selected_daily_survey.shape (14, 7)
weekly_survey.shape (14,)
selected_daily_survey.shape (26, 7)
weekly_survey.shape (26,)
selected_daily_survey.shape (13, 7)
weekly_survey.shape (13,)
selected_daily_survey.shape (22, 7)
weekly_survey.shape (22,)
selected_daily_survey.shape (32, 7)
weekly_survey.shape (32,)
selected_daily_survey.shape (28, 7)
weekly_survey.shape (28,)


In [53]:
print(len(reliable_train_ds_lst))

60


In [54]:
print(user_lst)

['direct_109', 'direct_116', 'direct_143', 'direct_159', 'direct_167', 'direct_263', 'direct_264', 'direct_287', 'direct_301', 'direct_377', 'direct_391', 'direct_393', 'direct_413', 'direct_417', 'direct_455', 'direct_465', 'direct_47', 'direct_525', 'sema4_101', 'sema4_1055', 'sema4_1056', 'sema4_1060', 'sema4_1099', 'sema4_118', 'sema4_131', 'sema4_138', 'sema4_15', 'sema4_179', 'sema4_200', 'sema4_249', 'sema4_251', 'sema4_255', 'sema4_281', 'sema4_331', 'sema4_383', 'sema4_412', 'sema4_415', 'sema4_433', 'sema4_442', 'sema4_489', 'sema4_492', 'sema4_501', 'sema4_524', 'sema4_54', 'sema4_591', 'sema4_595', 'sema4_613', 'sema4_66', 'sema4_717', 'sema4_720', 'sema4_773', 'sema4_777', 'sema4_783', 'sema4_842', 'sema4_863', 'sema4_880', 'sema4_89', 'sema4_966', 'sema4_97', 'sema4_990']


In [37]:
print(np.sum(np.array(corr_lst) > 0.5))
print(np.sum(np.array(icc_lst) > 0.5))

0
0


All average daily vs. weekly, corr > 0: 365 particiapnts\
3 days average daily vs. weeikly, corr > 0: 346 participants\
so take average of all 7 days

In [38]:
train_set = ConcatDataset(train_set_lst)
# test_set = ConcatDataset(reliable_test_ds_lst)
train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True)
# test_dataloader = DataLoader(test_set, batch_size=128, shuffle=True)

In [39]:

# Initialize empty lists to store extracted features and labels
all_features = []
all_labels = []

# Extract features and labels from DataLoader and convert to NumPy arrays

# for dataloader in [train_dataloader, test_dataloader]:
for dataloader in [train_dataloader]:
    for batch_features, batch_labels, user_id in dataloader:
        all_features.append(batch_features.numpy().squeeze())
        all_labels.append(batch_labels.numpy().squeeze())

# Concatenate all batches to get the complete feature and label arrays
X_numpy = np.concatenate(all_features, axis=0)
y_numpy = np.concatenate(all_labels, axis=0)
# X_numpy = np.concatenate(all_features, axis=0)
# y_numpy = np.concatenate(all_labels, axis=0)

# print(X_numpy.shape)
# print(y_numpy.shape)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# from sklearn.svm import SVR
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

# # Initialize and train an SVM model

# # svm_model = SVR()
# svm_model = MLPRegressor()
# svm_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = svm_model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)

# print("mae", mae)
# print("r2", r2)
# print("mse", mse)

In [ ]:
X_numpy.shape

In [ ]:
y_numpy.shape

In [ ]:
my_idx = list(range(0, y_numpy.shape[0]))

In [ ]:
df = pd.DataFrame({'daily': np.sum(X_numpy, axis=1), 'weekly': y_numpy, 'target': my_idx})


In [ ]:
melted_df = pd.melt(df, id_vars='target', var_name='y_label', value_name='y_value')
icc = pg.intraclass_corr(data=melted_df, targets='target', raters='y_label', ratings='y_value')

icc.set_index('Type')
icc

In [ ]:
icc.loc[2, 'ICC']


don't seem to be a proper way of aggregating all reader's response, i should do icc for each participants\\
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4913118/ suggests doing two-way mixed effect icc, which should be ICC3\\
single fixed raters 0.16 after filtering corr > 0, disable mental\
single fixed raters 0.14 after filtering corr > 0, disable physical\
single fixed raters 0.16 after filtering corr > 0, disable none\

single fixed raters 0.21 after filtering corr > 0.5, disable none
single fixed raters 0.

single fixed raters 0.14 without filtering, disable mental\
single fixed raters 0.11 without filtering, disable physical\
single fixed raters 0.13 without filtering, disable none\

Let's do edema separately since we don't have enough datapoint for each people\
